# Lesson 6.3: Error Handling and Debugging

---

In the process of developing any software application, especially with complex systems like LLM applications using LangChain, encountering errors is inevitable. Understanding how these errors arise and knowing how to handle and debug them are crucial skills. This lesson will focus on common errors, error handling techniques, and effective debugging tools in LangChain.

## 1. Common Errors in LLM and LangChain Application Development

### 1.1. API Rate Limits

* **Concept:** LLM providers (like OpenAI, Google) often impose limits on the number of requests or tokens you can send within a certain time frame (e.g., requests per minute, tokens per minute).
* **Causes:** Sending too many requests in a short period, or sending excessively large requests.
* **Consequences:** Requests are rejected with an HTTP 429 (Too Many Requests) error.
* **Solutions:**
    * **Exponential Backoff:** Automatically retry the request after increasing delays.
    * **Rate Limiting:** Limit the number of requests from your application's side.
    * **Upgrade API Plan:** If you frequently hit limits, consider upgrading your service plan.

### 1.2. Invalid Prompts

* **Concept:** Errors occur when the prompt sent to the LLM does not conform to the model's expected format or contains invalid characters/structures.
* **Causes:**
    * Syntax errors in `f-string` or `template`.
    * Missing variables in the prompt.
    * Prompt is too long, exceeding the LLM's context window.
    * Using disallowed special characters.
* **Consequences:** LLM returns an error, or a irrelevant/nonsensical response.
* **Solutions:** Carefully check prompt syntax, ensure all variables are populated, check prompt length.

### 1.3. Parsing Errors

* **Concept:** Occurs when the LLM's output cannot be parsed into the desired format by an `OutputParser` or by the `AgentExecutor` (when an Agent attempts to call a Tool).
* **Causes:**
    * LLM generates a response that is not valid JSON, XML, or the Agent's action format.
    * LLM `temperature` is too high, causing it to generate creative but unstructured output.
    * The prompt is not clear enough in instructing the LLM about the output format.
* **Consequences:** Application crashes with a parsing error, or the Agent cannot perform the next action.
* **Solutions:**
    * **Prompt Engineering:** Ask the LLM to output a specific format, provide few-shot examples.
    * **Lower `temperature`:** When structured output is required.
    * **`handle_parsing_errors=True`:** In `AgentExecutor`, allows the Agent to attempt self-correction.
    * **Custom Output Parser:** Write a custom parser to handle variations in output.

### 1.4. Connection Errors

* **Concept:** Network errors that occur when the application cannot connect to the LLM's API or external Tool APIs.
* **Causes:** Loss of internet connection, firewall blocking, API server downtime, incorrect API URL.
* **Consequences:** Request fails with a connection error.
* **Solutions:** Check network connectivity, ensure correct API URLs, check API server status.


---

## 2. Common Error Handling Technique: try-except blocks

The most basic and effective way to handle errors in Python is by using `try-except` blocks. This allows you to "catch" exceptions and perform graceful recovery actions or error notifications.

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from openai import OpenAIError # To catch specific OpenAI errors

# Thiết lập biến môi trường cho khóa API của OpenAI
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)
prompt_template = PromptTemplate.from_template("Viết một câu nói truyền cảm hứng về {topic}.") # Write an inspiring quote about {topic}.

def generate_inspiration(topic: str):
    try:
        # Giả lập một lỗi API (ví dụ: do sai khóa API hoặc rate limit)
        # Trong thực tế, lỗi này sẽ do thư viện OpenAI ném ra
        if topic == "lỗi": # error
            raise OpenAIError("Simulated OpenAI API error: Invalid API key or Rate Limit Exceeded.")

        response = llm.invoke(prompt_template.format(topic=topic))
        print(f"Thành công: {response.content}") # Success:
    except OpenAIError as e:
        print(f"Lỗi API OpenAI: {e}. Vui lòng kiểm tra khóa API hoặc giới hạn tốc độ.") # OpenAI API Error: {e}. Please check API key or rate limit.
    except Exception as e:
        print(f"Một lỗi không xác định đã xảy ra: {e}") # An unknown error occurred:

print("--- Thực hành try-except blocks ---") # --- Practical try-except blocks ---
generate_inspiration("hạnh phúc") # happiness
generate_inspiration("lỗi") # error (will trigger simulated error)
generate_inspiration("thành công") # success
print("-" * 30)


---

## 3. Using Debugging Tools

### 3.1. Print Statements

A classic yet still very effective debugging method. By adding `print()` statements at various points in your code, you can track variable values, execution flow, and pinpoint where errors occur.

In [ ]:
# Example already illustrated in previous lessons and will continue in the practical section.
# print(f"Giá trị biến X: {X}") # Value of variable X:


### 3.2. Using `verbose=True` in Agent Executor (and Chains)

LangChain provides a `verbose=True` parameter for most Chains and especially for `AgentExecutor`. When `verbose` is enabled, LangChain will print detailed intermediate steps of the execution process, including:

* **Thoughts:** What the LLM is thinking.
* **Actions:** Which tool the LLM decided to call with what input.
* **Observations:** The result returned from the tool execution.
* **Final Answer:** When the Agent concludes.

This is incredibly useful for understanding why an Agent made a particular decision or why it failed.

In [ ]:
# Example will be illustrated in the practical section.
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### 3.3. LangSmith (Overview)

**LangSmith** is a platform developed by LangChain, providing powerful tools for **monitoring, debugging, testing, and evaluating** LLM applications.

* **Key Features:**
    * **Trace Visualization:** Visually displays the entire execution flow of a Chain/Agent, including all LLM calls, Tool calls, and intermediate steps. Helps you easily identify bottlenecks or errors.
    * **Debugging:** Provides detailed information about inputs, outputs, and errors for each step.
    * **Monitoring:** Tracks application performance, costs, latency, and error rates in real-time.
    * **Evaluation:** Supports automated testing and evaluation of different application versions.
    * **Dataset Management:** Manages datasets for testing and fine-tuning.
* **Usage:** You need to sign up for a LangSmith account and set the environment variables `LANGCHAIN_TRACING_V2=true`, `LANGCHAIN_API_KEY`, `LANGCHAIN_PROJECT`. After that, LangChain will automatically send traces to LangSmith.




---

## 4. Practical Exercise: Handling Error Scenarios and Debugging Applications

We will practice common error scenarios and how to debug them.

**Preparation:**
* Ensure you have `langchain-openai`, `google-search-results`, `numexpr` installed.
* Set the `OPENAI_API_KEY` and `SERPAPI_API_KEY` environment variables.

In [ ]:
# Install libraries if not already installed
# pip install langchain-openai openai google-search-results numexpr

import os
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.utilities import SerpAPIWrapper
from langchain_community.tools import Tool
from langchain_community.tools.calculator.tool import Calculator
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.exceptions import OutputParserException # To catch parsing errors
from typing import Any, Dict, List, Optional, Union
from langchain_core.outputs import LLMResult

# Thiết lập biến môi trường cho khóa API của OpenAI và SerpAPI
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
# os.environ["SERPAPI_API_KEY"] = "YOUR_SERPAPI_API_KEY"

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Khởi tạo Tools
search_tool = Tool(
    name="Google Search",
    func=SerpAPIWrapper().run,
    description="Hữu ích khi bạn cần tìm kiếm thông tin trên Google về các sự kiện hiện tại hoặc dữ liệu thực tế." # Useful when you need to search for information on Google about current events or factual data.
)
calculator_tool = Calculator()
tools = [search_tool, calculator_tool]

# Định nghĩa Prompt cho Agent
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "Bạn là một trợ lý hữu ích. Bạn có quyền truy cập vào các công cụ sau: {tools}. Sử dụng chúng để trả lời các câu hỏi."), # You are a helpful assistant. You have access to the following tools: {tools}. Use them to answer questions.
    MessagesPlaceholder(variable_name="agent_scratchpad"),
    ("human", "{input}"),
])

# Tạo Agent
agent = create_react_agent(llm, tools, agent_prompt)

# --- 4.1. Thực hành xử lý lỗi API (giả lập) ---
print("--- Thực hành xử lý lỗi API (giả lập) ---") # --- Practical API Error Handling (simulated) ---

# Giả lập một LLM có thể gây lỗi
class MockErrorLLM(ChatOpenAI):
    def _generate(self, messages: List[BaseMessage], stop: Optional[List[str]] = None, **kwargs: Any) -> LLMResult:
        # Giả lập lỗi API sau một số lần gọi
        if getattr(self, '_call_count', 0) < 1: # Lần gọi đầu tiên thành công
            setattr(self, '_call_count', getattr(self, '_call_count', 0) + 1)
            return super()._generate(messages, stop, **kwargs)
        else:
            setattr(self, '_call_count', getattr(self, '_call_count', 0) + 1)
            # Giả lập lỗi API
            raise Exception("Simulated API connection error or rate limit exceeded.")

mock_llm = MockErrorLLM(model_name="gpt-3.5-turbo", temperature=0)
mock_llm_chain = LLMChain(llm=mock_llm, prompt=PromptTemplate.from_template("Nói xin chào.")) # Say hello.

try:
    print("Thử gọi LLM lần 1:") # Attempting LLM call 1:
    mock_llm_chain.invoke({})
    print("Gọi LLM lần 1 thành công.") # LLM call 1 successful.
    
    print("\nThử gọi LLM lần 2 (sẽ lỗi):") # Attempting LLM call 2 (will fail):
    mock_llm_chain.invoke({})
    print("Gọi LLM lần 2 thành công (không mong đợi).") # LLM call 2 successful (unexpected).
except Exception as e:
    print(f"Đã bắt được lỗi khi gọi LLM: {e}") # Caught error when calling LLM:
print("-" * 30)

# --- 4.2. Thực hành xử lý lỗi phân tích cú pháp của Agent ---
print("\n--- Thực hành xử lý lỗi phân tích cú pháp của Agent ---") # --- Practical Agent Parsing Error Handling ---

# Tạo một Agent Executor với handle_parsing_errors=True
# Để minh họa lỗi, chúng ta sẽ tạo một LLM giả lập đôi khi trả về định dạng sai
class MalformedOutputLLM(ChatOpenAI):
    def _generate(self, messages: List[BaseMessage], stop: Optional[List[str]] = None, **kwargs: Any) -> LLMResult:
        # Sau một số lần gọi, trả về output không đúng định dạng tool call
        if getattr(self, '_malform_count', 0) < 1:
            setattr(self, '_malform_count', getattr(self, '_malform_count', 0) + 1)
            return super()._generate(messages, stop, **kwargs)
        else:
            setattr(self, '_malform_count', getattr(self, '_malform_count', 0) + 1)
            # Trả về một chuỗi không phải JSON hoặc không đúng định dạng Action
            # Điều này sẽ gây ra lỗi phân tích cú pháp trong AgentExecutor
            malformed_response = "Đây là một phản hồi không đúng định dạng mà Agent không thể hiểu." # This is a malformed response that the Agent cannot understand.
            return LLMResult(generations=[[BaseMessage(content=malformed_response)]])

malformed_llm = MalformedOutputLLM(model_name="gpt-3.5-turbo", temperature=0)
malformed_agent = create_react_agent(malformed_llm, tools, agent_prompt)

# Agent Executor với handle_parsing_errors=True
malformed_agent_executor = AgentExecutor(
    agent=malformed_agent,
    tools=tools,
    verbose=True, # Bật verbose để xem Agent cố gắng tự sửa lỗi
    handle_parsing_errors=True, # Cho phép Agent xử lý lỗi phân tích cú pháp
    max_iterations=3 # Giới hạn số lần lặp để tránh vòng lặp vô hạn
)

try:
    print("Thử chạy Agent với đầu ra LLM không đúng định dạng:") # Attempting to run Agent with malformed LLM output:
    response_malformed = malformed_agent_executor.invoke({"input": "Tìm kiếm thời tiết ở New York."}) # Search for weather in New York.
    print(f"Phản hồi từ Agent: {response_malformed['output']}") # Response from Agent:
except OutputParserException as e:
    print(f"Đã bắt được lỗi phân tích cú pháp (nhưng AgentExecutor đã cố gắng xử lý): {e}") # Caught parsing error (but AgentExecutor attempted to handle):
except Exception as e:
    print(f"Một lỗi khác đã xảy ra: {e}") # Another error occurred:
print("-" * 30)

# --- 4.3. Debugging luồng Agent với `verbose=True` ---
print("\n--- Debugging luồng Agent với verbose=True ---") # --- Debugging Agent flow with verbose=True ---

# Tạo Agent Executor với verbose=True
debug_agent_executor = AgentExecutor(
    agent=agent, # Sử dụng agent bình thường
    tools=tools,
    verbose=True, # BẬT VERBOSE
    max_iterations=5 # Giới hạn số lần lặp
)

print("Yêu cầu: Tính tổng 123 và 456, sau đó tìm kiếm thông tin về LangChain.") # Request: Calculate the sum of 123 and 456, then search for information about LangChain.
response_debug = debug_agent_executor.invoke({"input": "Tính tổng 123 và 456, sau đó tìm kiếm thông tin về LangChain."}) # Calculate the sum of 123 and 456, then search for information about LangChain.
print(f"\nPhản hồi cuối cùng từ Agent: {response_debug['output']}") # Final response from Agent:
print("-" * 30)

# Dọn dẹp (nếu có file cache SQLite)
# if os.path.exists(".langchain.db"):
#     os.remove(".langchain.db")

**Explanation of practical sections:**

* **Simulated API Error Handling:** We define `MockErrorLLM` to simulate an LLM that occasionally throws an error. The `try-except` block surrounding the LLM call catches this error and prints a friendly message instead of crashing the application.
* **Agent Parsing Error Handling:**
    * `MalformedOutputLLM` is created to simulate an LLM that sometimes generates malformed output that the `AgentExecutor` cannot parse into an `AgentAction` or `AgentFinish`.
    * When `AgentExecutor` is initialized with `handle_parsing_errors=True`, instead of stopping immediately, it passes the parsing error back to the LLM as an "Observation." The LLM then attempts to self-correct in the next turn.
    * You will see in the `verbose=True` output that the Agent receives an "Observation" about the parsing error and tries to re-think to generate a valid action.
* **Debugging Agent Flow with `verbose=True`:**
    * When you run `debug_agent_executor.invoke` with `verbose=True`, you will see each "Thought," "Action," and "Observation" of the Agent.
    * This allows you to trace the LLM's reasoning step-by-step, which tool was called, and the result of that tool. It's crucial for understanding why the Agent behaves the way it does and for identifying issues in your logic or prompt.


---

## Lesson Summary

This lesson equipped you with essential skills for error handling and debugging in LLM application development with LangChain. You learned about **common errors** such as API rate limits, invalid prompts, parsing errors, and connection errors. We explored the **`try-except`** technique for graceful error handling. For **debugging**, you learned how to use tools like **print statements** and especially the **`verbose=True`** parameter in `AgentExecutor` to trace the detailed flow of Agent activity. Finally, you were introduced to **LangSmith**, a powerful platform for in-depth monitoring and debugging. Through practical examples, you applied these techniques to handle error scenarios and gain a better understanding of how to debug your LangChain applications.